In [2]:
import os
import sys
import tarfile
import time
import urllib.request

source = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
target = 'aclImdb_v1.tar.gz'

if os.path.exists(target):
    os.remove(target)


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024. ** 2 * duration)
    percent = count * block_size * 100. / total_size

    sys.stdout.write(f'\r{int(percent)}% | {progress_size / (1024. ** 2):.2f} MB '
                     f'| {speed:.2f} MB/s | {duration:.2f} sec elapsed')
    sys.stdout.flush()


if not os.path.isdir('aclImdb') and not os.path.isfile('aclImdb_v1.tar.gz'):
    urllib.request.urlretrieve(source, target, reporthook)

In [3]:
if not os.path.isdir('aclImdb'):

    with tarfile.open(target, 'r:gz') as tar:
        tar.extractall()

In [4]:
#assemble the text documents from decompressed download into a single csv file and put into a pandas dataframe
import pyprind
import pandas as pd
import os
import sys
from packaging import version


# change the `basepath` to the directory of the
# unzipped movie dataset

basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}

# if the progress bar does not show, change stream=sys.stdout to stream=2
pbar = pyprind.ProgBar(50000, stream=sys.stdout)

df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file),
                      'r', encoding='utf-8') as infile:
                txt = infile.read()

            if version.parse(pd.__version__) >= version.parse("1.3.2"):
                x = pd.DataFrame([[txt, labels[l]]], columns=['review', 'sentiment'])
                df = pd.concat([df, x], ignore_index=False)

            else:
                df = df.append([[txt, labels[l]]],
                               ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:21


In [5]:
#shuffling dataframe
import numpy as np


if version.parse(pd.__version__) >= version.parse("1.3.2"):
    df = df.sample(frac=1, random_state=0).reset_index(drop=True)

else:
    np.random.seed(0)
    df = df.reindex(np.random.permutation(df.index))

In [6]:
#save as csv file
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [7]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')

# the following is necessary on some computers:
df = df.rename(columns={"0": "review", "1": "sentiment"})

df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [8]:
df.shape

(50000, 2)

In [9]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [10]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [11]:
print(bag)

  (0, 6)	1
  (0, 4)	1
  (0, 1)	1
  (0, 3)	1
  (1, 6)	1
  (1, 1)	1
  (1, 8)	1
  (1, 5)	1
  (2, 6)	2
  (2, 4)	1
  (2, 1)	3
  (2, 3)	1
  (2, 8)	1
  (2, 5)	1
  (2, 0)	2
  (2, 2)	2
  (2, 7)	1


In [12]:
print(bag.toarray()) #each row in the matrix represents a sentence and each columns represents a word, with the count amount

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [13]:
np.set_printoptions(precision=2)

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs))
      .toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [15]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [16]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) #remove html markup
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text) #find emoticons
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', '')) #remove all non word characters (,./) and converted everything to lowercase and remove any 'nose' (-) in emoji
    return text

In [17]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [18]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [19]:
df['review'] = df['review'].apply(preprocessor) #appling preprocessor function to all rows

In [20]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [21]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [22]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [23]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/haydenlabrie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')
 if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [25]:
#split training and test data by 50/50
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [26]:
#Gridsearch for hyperparameter tuning using 5 stratified cross validation
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None) #we replaced CountVectorizer and TfidfTransformer from the previous subsection with TfidfVectorizer, which combines CountVectorizer with the TfidfTransformer.

"""
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]
"""

small_param_grid = [{'vect__ngram_range': [(1, 1)],
                     'vect__stop_words': [None],
                     'vect__tokenizer': [tokenizer, tokenizer_porter],
                     'clf__penalty': ['l2'],
                     'clf__C': [1.0, 10.0]},
                    {'vect__ngram_range': [(1, 1)],
                     'vect__stop_words': [stop, None],
                     'vect__tokenizer': [tokenizer],
                     'vect__use_idf':[False],
                     'vect__norm':[None],
                     'clf__penalty': ['l2'],
                  'clf__C': [1.0, 10.0]},
              ]

lr_tfidf = Pipeline([['vect', tfidf],
                     ['clf', LogisticRegression(solver='liblinear')]])

gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [27]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/Users/haydenlabrie/pytorch-test/env/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[['vect',
                                        TfidfVectorizer(lowercase=False)],
                                       ['clf',
                                        LogisticRegression(solver='liblinear')]]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x144613dc0>,
                                              <function tokenizer_porter at 0x144613e50>]},
                         {'...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
                                                'his', 'himself', 'she',
                                                "she's", 'her', 'hers',
                                                'herself', 'it', "it's", 'its',
                                                'itself', ...],
                                               None],
                          'vect__tokenizer': [<function tokenizer at 0x144613dc0>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=1)

In [28]:
print(f'Best parameter set: {gs_lr_tfidf.best_params_}')
print(f'CV Accuracy: {gs_lr_tfidf.best_score_:.3f}')

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x144613dc0>}
CV Accuracy: 0.897


In [29]:
print(tfidf.fit_transform(X_train))

  (0, 5403)	0.0612187943655873
  (0, 9382)	0.07643633441938372
  (0, 69923)	0.047425811555284164
  (0, 61492)	0.06621059773392657
  (0, 74816)	0.06909114350603188
  (0, 46493)	0.025580709369178882
  (0, 20823)	0.0939861962942094
  (0, 22535)	0.06324376399162435
  (0, 38920)	0.05123422253766132
  (0, 69227)	0.021956350140750574
  (0, 50430)	0.07825997381094954
  (0, 17509)	0.04811621213144935
  (0, 39343)	0.03949635931911951
  (0, 62366)	0.042595186987595964
  (0, 60680)	0.056899526808208784
  (0, 16268)	0.05702373354502255
  (0, 32120)	0.07813746565778833
  (0, 33101)	0.02849244470969925
  (0, 19547)	0.11367850069840753
  (0, 1292)	0.04814586321881018
  (0, 51390)	0.12462308057368808
  (0, 15337)	0.08525615413746955
  (0, 63838)	0.11978002682670494
  (0, 33112)	0.03770455812789957
  (0, 69049)	0.027256523077415683
  :	:
  (25000, 56090)	0.08134017524444206
  (25000, 63908)	0.021588289001418508
  (25000, 69227)	0.04513548983792351
  (25000, 33112)	0.03875447625210424
  (25000, 69049)	0.

In [30]:
tfidf.fit_transform(X_train).toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
clf = gs_lr_tfidf.best_estimator_
print(f'Test Accuracy: {clf.score(X_test, y_test):.3f}')

Test Accuracy: 0.899


In [93]:
import numpy as np
import re
from nltk.corpus import stopwords


# The `stop` is defined as earlier in this chapter
# Added it here for convenience, so that this section
# can be run as standalone without executing prior code
# in the directory
stop = stopwords.words('english')


def tokenizer(text): #seperates words into tokens and removes stop words; stop words are common words that dont give much value like 'but' 'this'
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized


def stream_docs(path): #reads in and returns one document at a time
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [94]:
next(stream_docs(path='movie_data.csv')) #To verify that our stream_docs function works correctly, let’s read in the first document from the movie_data.csv file, which should return a tuple consisting of the review text as well as the corresponding class label

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [95]:
def get_minibatch(doc_stream, size): #take a document stream from the stream_docs function and return a particular number of documents specified by the size parameter
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [96]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

#another useful vectorizer for text processing implemented in scikit-learn is HashingVectorizer
vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer) #uses tokenizer function created above

In [97]:
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

clf = SGDClassifier(loss='log_loss', random_state=1) #using logistic regression to classify rating of good or bad review


doc_stream = stream_docs(path='movie_data.csv') #where the data is coming from

In [98]:
#We initialized the progress bar object with 45 iterations and, in the following for loop, we iterated over 45 mini-batches of documents where each mini-batch consists of 1,000 documents.
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

print(X_train)

  (0, 6895)	-0.15762208124782012
  (0, 9568)	0.07881104062391006
  (0, 178869)	0.07881104062391006
  (0, 224349)	-0.07881104062391006
  (0, 229022)	-0.07881104062391006
  (0, 229844)	-0.07881104062391006
  (0, 242292)	0.07881104062391006
  (0, 264412)	0.07881104062391006
  (0, 270295)	-0.07881104062391006
  (0, 340157)	0.07881104062391006
  (0, 378379)	-0.07881104062391006
  (0, 434764)	0.07881104062391006
  (0, 481428)	0.07881104062391006
  (0, 518663)	-0.07881104062391006
  (0, 523058)	-0.15762208124782012
  (0, 530893)	-0.23643312187173018
  (0, 543078)	0.15762208124782012
  (0, 546484)	-0.15762208124782012
  (0, 559973)	-0.07881104062391006
  (0, 597420)	0.07881104062391006
  (0, 606863)	-0.07881104062391006
  (0, 639494)	-0.07881104062391006
  (0, 659316)	-0.07881104062391006
  (0, 663420)	-0.07881104062391006
  (0, 687808)	-0.07881104062391006
  :	:
  (999, 1124858)	-0.10540925533894598
  (999, 1132800)	-0.10540925533894598
  (999, 1155574)	-0.10540925533894598
  (999, 1235946)	0

In [99]:
X_test, y_test = get_minibatch(doc_stream, size=5000) #Having completed the incremental learning process, we will use the last 5,000 documents to evaluate the performance of our model
X_test = vect.transform(X_test)
print(f'Accuracy: {clf.score(X_test, y_test):.3f}')

Accuracy: 0.868


In [100]:
clf = clf.partial_fit(X_test, y_test) #Finally, we can use the last 5,000 documents to update our model

In [101]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')

# the following is necessary on some computers:
df = df.rename(columns={"0": "review", "1": "sentiment"})

df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [102]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1, #Notice that we set the maximum document frequency of words to be considered to 10 percent (max_ df=.1) to exclude words that occur too frequently across documents
                        max_features=5000) #Also, we limited the number of words to be considered to the most frequently occurring 5,000 words (max_features=5000), to limit the dimensionality of this dataset to improve the inference performed by LDA
X = count.fit_transform(df['review'].values)

In [104]:
print(X)

  (0, 4450)	2
  (0, 2782)	2
  (0, 2738)	1
  (0, 1973)	1
  (0, 2959)	1
  (0, 2130)	1
  (0, 841)	1
  (0, 291)	1
  (0, 2088)	1
  (0, 3046)	1
  (0, 1573)	1
  (0, 2043)	1
  (0, 2973)	1
  (0, 2208)	1
  (0, 2972)	5
  (0, 3649)	1
  (0, 2566)	1
  (0, 4976)	1
  (0, 2773)	2
  (0, 818)	1
  (0, 2539)	1
  (0, 1266)	3
  (0, 1675)	1
  (0, 4037)	1
  (0, 4603)	1
  :	:
  (49998, 2512)	1
  (49998, 4807)	1
  (49998, 3731)	1
  (49998, 2627)	1
  (49998, 2514)	1
  (49998, 556)	1
  (49998, 4598)	1
  (49998, 4253)	1
  (49998, 4660)	1
  (49998, 4187)	1
  (49998, 3539)	1
  (49998, 1619)	1
  (49998, 3739)	2
  (49998, 4646)	1
  (49999, 1387)	1
  (49999, 1500)	1
  (49999, 4491)	1
  (49999, 1291)	1
  (49999, 2599)	1
  (49999, 1614)	1
  (49999, 2221)	1
  (49999, 2914)	1
  (49999, 601)	1
  (49999, 4905)	1
  (49999, 4808)	1


In [105]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch') #get 10 different categories
X_topics = lda.fit_transform(X)

In [106]:
lda.components_.shape #stores a matrix containing the word importance (here, 5000) for each of the 10 topics in increasing order

(10, 5000)

In [107]:
#finding top 5 words for each category
n_top_words = 5
feature_names = count.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {(topic_idx + 1)}:')
    print(' '.join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]]))
#topic1 -> Generally bad movies (not really a topic category)
#topic2 -> Movies about families
#topic3 -> War movies
#topic4 -> Art movies
#5 -> Crime movies
#6 -> Horror movies
#7 -> Comedy movie reviews
#8 -> Movies somehow related to TV shows
#9 -> Movies based on books
#10 -> Action movies

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode war episodes tv
Topic 9:
book version original read novel
Topic 10:
action fight guy guys cool


In [108]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print(f'\nHorror movie #{(iter_idx + 1)}:')
    print(df['review'][movie_idx][:300], '...')


Horror movie #1:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

Horror movie #2:
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

Horror movie #3:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...


In [142]:
print(horror[49999]) #closer to 0, more likely to belong to that category

25114


In [143]:
print(df['review'][25114][:300], '...')

Match 1: Tag Team Table Match Bubba Ray and Spike Dudley vs Eddie Guerrero and Chris Benoit Bubba Ray and Spike Dudley started things off with a Tag Team Table Match against Eddie Guerrero and Chris Benoit. According to the rules of the match, both opponents have to go through tables in order to get ...
